# 5. トレーニング設定

In [ ]:

# Start training
logging.info("Starting training...")
try:
    checkpoint_dir = MODEL_OUTPUT_DIR  
    resume_from_checkpoint = None
    
    # Check if running in Kaggle environment
    is_kaggle = os.path.exists('/kaggle/working')
    
    # Checkpoint status and processing
    if os.path.exists(checkpoint_dir):
        print("\nChecking checkpoint status...")  
        checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith("checkpoint-")]
        if checkpoints:
            # Get latest checkpoint
            latest_checkpoint = max(checkpoints, key=lambda x: int(x.split("-")[1]))
            checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint)
            print(f"Found latest checkpoint: {latest_checkpoint}") 
            
            # Check checkpoint status
            state_path = os.path.join(checkpoint_path, "trainer_state.json")
            if os.path.exists(state_path):
                with open(state_path, 'r') as f:
                    state = json.load(f)
                current_epoch = state.get('epoch', 0)
                print(f"\nCurrent training status:")  
                print(f"Current epoch: {current_epoch}")  
                print(f"Target epochs: {training_args.num_train_epochs}")  
                
                # Exit safely if completed
                if current_epoch >= training_args.num_train_epochs - 0.1:
                    print("\n" + "="*50)
                    print("IMPORTANT NOTICE:")
                    print(f"Training has already been completed at epoch {current_epoch}!")
                    print(f"Target epochs was {training_args.num_train_epochs}")  
                    print(f"Trained model is available at: {checkpoint_dir}")
                    print("="*50 + "\n")
                    logging.info("Training has already been completed. Exiting to protect existing model.")
                    logging.info(f"Trained model is available at: {checkpoint_dir}")
                    exit(0)
            else:
                logging.warning("Invalid checkpoint state found. Please check manually.")
                logging.warning(f"Checkpoint directory: {checkpoint_dir}")
                if not is_kaggle:  
                    user_input = input("Do you want to continue and overwrite? (yes/no): ")
                    if user_input.lower() != 'yes':
                        logging.info("Aborting to protect existing data.")
                        exit(0)
        else:
            logging.warning("Checkpoint directory exists but no checkpoints found.")
            if not is_kaggle:  
                user_input = input("Do you want to continue and overwrite the directory? (yes/no): ")
                if user_input.lower() != 'yes':
                    logging.info("Aborting to protect existing data.")
                    exit(0)

    # Start training (or resume)
    trainer.train(resume_from_checkpoint=resume_from_checkpoint)
    logging.info("Training completed successfully!")



<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

このコードについて、ソクラテス式チャットボットのトレーニングを例に説明させていただきます。

### 基本的な目的
このコードは、トレーニングの再開や既存モデルの保護を管理する部分です。トレーニングが途中で中断された場合に備えて、定期的にチェックポイント（進捗の保存）を作成し、必要に応じて途中から再開できるようにしています。

### 具体的な処理の流れ

1. **チェックポイントの確認**
```python
checkpoint_dir = MODEL_OUTPUT_DIR  
```
- まず、チェックポイントが保存されているフォルダを確認します
- 例：`models/kaggle_model_ver2/model`のような場所です

2. **既存のトレーニング状況チェック**
```python
checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith("checkpoint-")]
```
- フォルダ内にある`checkpoint-1000`、`checkpoint-2000`のような保存ポイントを探します
- これらは、例えば「ソクラテス式の質問の仕方を1000回学習した時点」「2000回学習した時点」などの記録です

3. **最新の進捗確認**
```python
latest_checkpoint = max(checkpoints, key=lambda x: int(x.split("-")[1]))
```
- 最も新しいチェックポイントを見つけます
- 例：`checkpoint-3000`が最新だった場合、これはソクラテスの対話スタイルを3000回学習した時点の記録となります

4. **トレーニング状況の判断**
```python
if current_epoch >= training_args.num_train_epochs - 0.1:
```
- 学習が既に完了しているかチェックします
- 例：30エポック（全データを30周学習）を目標にしていて、既に30エポック終わっている場合は、
  「ソクラテスの対話スタイル（質問の投げかけ方、考えを引き出す話し方など）を十分に学習済み」
  と判断して、新たな学習を開始しないようにします

5. **ユーザーへの確認**
```python
user_input = input("Do you want to continue and overwrite? (yes/no): ")
```
- 既存のデータがある場合、上書きしても良いか確認します
- 例：「既に良い感じにソクラテス式の対話ができるようになっているモデルがありますが、
  新しく最初から学習し直しますか？」というような確認です

6. **トレーニングの開始または再開**
```python
trainer.train(resume_from_checkpoint=resume_from_checkpoint)
```
- 実際のトレーニングを開始します
- 途中のチェックポイントがある場合：そこから再開（例：3000回目の学習から継続）
- ない場合：最初から開始（ソクラテス式の対話スタイルを一から学習）

### 重要なポイント
このコードの特に重要な役割は：
1. 学習の進捗を守る（誤って上書きしない）
2. 途中で中断しても再開できる
3. 既に十分な学習が完了している場合に、不要な再学習を防ぐ

これにより、ソクラテス式チャットボットの学習を効率的かつ安全に管理することができます。

        
</div>
    </details>
</div>


In [ ]:

    ### 5.2 モデル保存と設定エクスポート
    # Save settings (as JSON)
    def convert_to_serializable(obj):
        if isinstance(obj, set):
            return list(obj)
        elif isinstance(obj, dict):
            return {k: convert_to_serializable(v) for k, v in obj.items()}
        elif isinstance(obj, (list, tuple)):
            return [convert_to_serializable(x) for x in obj]
        return obj

    # Convert each setting
    training_args_dict = convert_to_serializable(training_args.to_dict())
    lora_config_dict = convert_to_serializable(lora_config.to_dict())

    config_dict = {
        "model_name": model_name,
        "training_args": training_args_dict,
        "lora_config": lora_config_dict,
        "bnb_config": {
            "load_in_4bit": bnb_config.load_in_4bit,
            "bnb_4bit_use_double_quant": bnb_config.bnb_4bit_use_double_quant,
            "bnb_4bit_quant_type": bnb_config.bnb_4bit_quant_type,
            "bnb_4bit_compute_dtype": str(bnb_config.bnb_4bit_compute_dtype),
        }
    }
    
    # Save configurations
    with open(os.path.join(training_args.output_dir, "training_config.json"), "w", encoding="utf-8") as f:
        json.dump(config_dict, f, indent=2, ensure_ascii=False)
    
    # Save model and settings
    trainer.save_model()
    model.config.save_pretrained(training_args.output_dir)
    tokenizer.save_pretrained(training_args.output_dir)
    logging.info("Model and configuration saved successfully!")

except Exception as e:
    logging.error(f"An error occurred: {str(e)}")
    raise


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

このコードについて、ソクラテス式チャットボットのプロジェクトを例に説明させていただきます。

### 基本的な目的
このコードは、学習が完了したモデルと、その学習に使用した設定を保存するための部分です。将来、同じような条件でモデルを再学習したり、モデルの設定を確認したりする際に必要となります。

### 具体的な処理の流れ

1. **データの保存形式への変換**
```python
def convert_to_serializable(obj):
    if isinstance(obj, set):
        return list(obj)
    # ...
```
- JSONファイルとして保存できる形式にデータを変換します
- 例：ソクラテス式の対話スタイルを学習する際の設定値（学習率、エポック数など）を保存可能な形式に変換

2. **学習設定の変換**
```python
training_args_dict = convert_to_serializable(training_args.to_dict())
lora_config_dict = convert_to_serializable(lora_config.to_dict())
```
- 学習に使用した設定を保存用の形式に変換します
- 例：
  - 「何回対話を繰り返して学習したか」
  - 「どのくらいの速さで学習したか」
  - 「ソクラテスらしい質問の仕方をどう学習させたか」などの設定

3. **設定のまとめ**
```python
config_dict = {
    "model_name": model_name,
    "training_args": training_args_dict,
    "lora_config": lora_config_dict,
    "bnb_config": {
        # 量子化（モデルの軽量化）の設定
    }
}
```
- すべての設定を一つの辞書にまとめます
- 例：
  - 使用したベースモデルの名前
  - トレーニングの詳細設定
  - メモリ使用の最適化設定など

4. **設定の保存**
```python
with open(os.path.join(training_args.output_dir, "training_config.json"), "w", encoding="utf-8") as f:
    json.dump(config_dict, f, indent=2, ensure_ascii=False)
```
- まとめた設定をJSONファイルとして保存します
- 例：`models/kaggle_model_ver2/model/training_config.json`として保存

5. **モデルと関連設定の保存**
```python
trainer.save_model()
model.config.save_pretrained(training_args.output_dir)
tokenizer.save_pretrained(training_args.output_dir)
```
- 学習済みモデル本体と関連設定を保存します
- 保存される内容：
  - ソクラテス式の対話の仕方を学習したモデル本体
  - 日本語を処理するための設定
  - 特殊な言い回しの処理方法など

6. **エラー処理**
```python
except Exception as e:
    logging.error(f"An error occurred: {str(e)}")
    raise
```
- 保存中に問題が発生した場合のエラー処理を行います

### 重要なポイント
このコードの特に重要な役割は：
1. 学習したモデルを再利用可能な形で保存
2. 学習設定を詳細に記録（再現性の確保）
3. 日本語処理に必要な設定も含めて保存

これにより：
- 同じ条件で再学習が可能
- モデルの改良時に以前の設定と比較可能
- チームでの共有や実験の再現が容易

例えば、「より自然なソクラテス式の対話」を目指して設定を調整する際に、以前の設定を参照できるようになります。

        
</div>
    </details>
</div>
